# NER для русского языка в spaCy 3: удобно и легко
[Ссылка на статью](https://vc.ru/dev/299248-ner-dlya-russkogo-yazyka-v-spacy-3-udobno-i-legko?ysclid=l4wqnn7rur405121406)

Славянские языки, в том числе и русский, считаются довольно сложными для обработки. В основном, из-за богатой системы окончаний, свободного порядка слов и других морфологических и синтаксических явлений. Распознавание именованных сущностей (далее, NER) представляется трудной задачей для славянских языков, где синтаксические зависимости часто маркируются морфологическими чертами, нежели определенным порядком словоформ. Поэтому NER сложен для этих языков, в сравнении с германскими или романскими языками.

NER – популярная задача в сфере обработки естественного языка. Она заключается в распознавании именованных сущностей в тексте и определение их типов.

За много лет подходы к NER для русского языка претерпели множество изменений. От rule-based тенденции, предложенной в 2004 году (Popov et al., 2004) до современных state-of-the-art решений с использованием encoder-decoder архитектуры и трансформеров. Появилось множество NLP библиотек, стремящихся к универсальности и простоте. На протяжении истории развития обработки языка, NLP инструментарии поддерживали несколько «основных» языков, но позже эта тенденция сместилась в сторону мультиязыковой обработки текста. Один из таких инструментариев – spaCy (бесплатная опенсоурс библиотека для усовершенствованной обработки естественного языка), который включает в себя предобученные модели для 18 разных языков. Для большинства задач spaCy (Honnibal & Montani, 2017) использует сверточные нейронные сети, но для NER используются transition-based подходы.

Зимой 2021 года у spaCy вышла новая версия – 3: появилась русская модель и возможность выбора стандартной модели для обучения в зависимости от наличия GPU. Для задачи NER для русского языка spaCy предлагает tok2vec и Multilingual BERT. Также, пользователь может самостоятельно выбрать предобученную модель из списка на HuggingFace.

Попробуем натренировать NER на собственном датасете с использованием двух предложенных моделей. BSNLP Shared Task дает возможность загрузить тренировочные данные с аннотацией для нескольких славянских языков. Загрузим данные 2021 года. Они представлены в виде файлов с текстом и соответствующих лейблов следующего вида:

Последний столбец нас не интересует, так как он относится к части задания Entity Matching, первый столбец – словоформа в тексте, второй – лемма, третий – именованная сущность. Чтобы работать с этими данными в spaCy, нужно привести их в специальный бинарный формат. Более подробней о нём – здесь.

### Data Processing
Сначала находим нужную сущность из файлов с лейблами в тексте и записываем формат [index.start(), index.end(), label] в список, где index – индекс буквы в тексте, а label – аннотированная сущность. Затем очищаем полученный список от возможных пересечений индексов. Далее, с помощью встроенных функций spaCy приводим наш список в следующий формат: [«O», «O», «U-LOC», «O»], а потом кладем тексты и полученные тэги в привычный для spaCy doc.

In [3]:
import spacy
import os
import pandas as pd

def make_docs(folder, doc_list):
    nlp = spacy.load('ru_core_news_lg')
    out = 'out'

    for filename in os.listdir(f'data/bsnlp2021_train_r1/raw/{folder}/ru'.format(folder=folder)):
        df = pd.read_csv(f'data/bsnlp2021_train_r1/annotated/{folder}/ru/{filename}{out}'.format(folder=folder, filename=filename[:-3], out='out'), skiprows=1, header=None, sep='\t', encoding='utf8',  error_bad_lines=False, engine='python')
        f = open('data/bsnlp2021_train_r1/raw/{folder}/ru/{filename}'.format(folder=folder,filename=filename), "r", encoding='utf8')
        list_words=df.iloc[:,0].tolist()
        labels = df.iloc[:,2].tolist()
        text = f.read()
        entities=[]
        for n in range(len(list_words)):
            for m in re.finditer(list_words[n].strip(), text):
                entities.append([m.start(), m.end(), labels[n]])

        for f in range(len(entities)):
            if len(entities[f])==3:
                for s in range(f+1, len(entities)):
                    if len(entities[s])==3 and len(entities[f])==3:
                        if entities[f][0]==entities[s][0] or entities[f][1]==entities[s][1]:
                            if (entities[f][1]-entities[f][0]) >= (entities[s][1]-entities[s][0]): 
                                entities.pop(s)
                                entities.insert(s, (''))
                            else:
                                entities.pop(f)
                                entities.insert(f, (''))
                        if len(entities[s])==3 and len(entities[f])==3:
                            if entities[f][0] in range(entities[s][0]+1, entities[s][1]):
                                entities.pop(f)
                                entities.insert(f, (''))
                            elif entities[s][0] in range(entities[f][0]+1, entities[f][1]):
                                entities.pop(s)
                                entities.insert(s, (''))

        entities_cleared = [i for i in entities if len(i)==3]
        doc = nlp(text)
        tags = offsets_to_biluo_tags(doc, entities_cleared)
        #assert tags == ["O", "O", "U-LOC", "O"]
        entities_x = biluo_tags_to_spans(doc, tags)
        doc.ents = entities_x
        doc_list.append(doc)